# pyengine Demo

This notebook demonstrates how to use the `pyengine` library for routing prototypes. This library provides a clean Python implementation of the core routing algorithms used in the C++ engine.

In [1]:
import sys
import os
# Add pyengine to the python path
sys.path.append(os.path.abspath(".."))

from pyengine import Router
import folium

## Initialize Router

We load the `burnaby` dataset from its DuckDB database. The `Router` class will automatically load the adjacency lists and shortcut metadata.

In [2]:
db_path = "../../../data/Burnaby.db"
router = Router(db_path)
print("Router initialized and data loaded.")

Router initialized and data loaded.


## Router API & Options

The `router.route_by_coordinates()` method supports several parameters:

### Supported Shortest Path Algorithms

| Algorithm | Description |
| :--- | :--- |
| `"classic"` | **(Default)** Bidirectional Hierarchical search (`bi_classic_sp`). |
| `"bi_lca"` | Bidirectional LCA-targeted search (Optimized hierarchical, `bi_lca_sp`). |
| `"bi_lca_res"`| Bidirectional resolution-pruned search (`bi_lca_res_sp`, alias `"pruned"`). |
| `"uni_lca"` | Unidirectional LCA-targeted search via phase-based state machine. |
| `"bi_dijkstra"`| Standard Bidirectional Dijkstra (no pruning, no filtering). |
| `"uni_dijkstra"`| Standard Unidirectional Dijkstra (baseline ground truth). |
| `"m2m"` | Many-to-Many classic bidirectional search. |

### Routing Parameters

| Option | Type | Default | Description |
| :--- | :--- | :--- | :--- |
| `algorithm` | `str` | `"classic"` | Selects which algorithm to use for the **shortest path**. |
| `include_alternative` | `bool` | `False` | If True, finds an alternative path relative to the shortest path. |
| `penalty_factor` | `float` | `2.0` | Penalty applied to shortest path edges during alternative search. |
| `max_candidates` | `int` | `5` | (M2M only) Number of nearest edge candidates for each point. |


## Combined Route Query

Find both the shortest part (using `bi_lca`) and an alternative path.

In [3]:
# Coordinates in Burnaby
source_coords = (49.23497, -123.00962)
target_coords = (49.26000, -122.94000)

result = router.route_by_coordinates(
    source_coords[0], source_coords[1],
    target_coords[0], target_coords[1],
    algorithm="bi_lca",
    include_alternative=True,
    penalty_factor=5.0
)

if result["success"]:
    print(f"Algorithm used for Shortest Path: {result.get('algorithm', 'bi_lca')}")
    print(f"Shortest Distance: {result['distance']:.2f}")
    print(f"Shortest Path Base Edges: {len(result['path'])}")
    
    if "alternative_route" in result:
        alt = result["alternative_route"]
        print(f"Alternative Distance: {alt['distance']:.2f}")
        print(f"Alternative Path Base Edges: {len(alt['path'])}")
else:
    print("Error:", result.get("error"))

Algorithm used for Shortest Path: bi_lca
Shortest Distance: 165.57
Shortest Path Base Edges: 95
Alternative Distance: 194.23
Alternative Path Base Edges: 110


In [8]:
result.keys()

dict_keys(['success', 'distance', 'path', 'shortcut_path', 'alternative_route'])

In [10]:
result['alternative_route'].keys()

dict_keys(['distance', 'path', 'shortcut_path'])

## Many-to-Many Routing

The `route_m2m_by_coordinates()` method finds the top K candidates for each coordinate and then finds the best overall path between any of these candidates.

In [4]:
m2m_result = router.route_m2m_by_coordinates(
    source_coords[0], source_coords[1],
    target_coords[0], target_coords[1],
    include_alternative=True,
    penalty_factor=5.0,
    max_candidates=3
)

if m2m_result["success"]:
    print(f"M2M Best Distance: {m2m_result['distance']:.2f}")
    print(f"M2M Path Base Edges: {len(m2m_result['path'])}")
else:
    print("M2M Error:", m2m_result.get("error"))

M2M Best Distance: 164.69
M2M Path Base Edges: 94


In [7]:
m2m_result.keys()

dict_keys(['success', 'distance', 'path', 'shortcut_path', 'alternative_route'])

## Baseline Ground Truth

Compare the result of unidirectional Dijkstra against the hierarchical search.

In [5]:
baseline = router.route_by_coordinates(
    source_coords[0], source_coords[1],
    target_coords[0], target_coords[1],
    algorithm="uni_dijkstra"
)

if baseline["success"]:
    print(f"Baseline (uni_dijkstra) Distance: {baseline['distance']:.2f}")
    print(f"Difference vs bi_lca: {result['distance'] - baseline['distance']:.6f}")

Baseline (uni_dijkstra) Distance: 160.37
Difference vs bi_lca: 5.207164
